In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio


In [ ]:
def make_gray_image(image):
    gray_image = image
    if len(image.shape) == 3:
        gray_image = np.mean(image, axis=2).astype(np.uint8)
    return gray_image

In [ ]:
def add_gaussian_noise(image, mean, std):
    noisy_image = np.copy(image).astype(np.uint8)
    noise = np.random.normal(mean, std, image.shape)
    noisy_image = noisy_image + noise
    noisy_image = ((noisy_image / np.max(noisy_image)) * 255)
    return noisy_image

In [ ]:
def D(image_size):
    kernel = np.zeros((image_size, image_size))
    center = image_size // 2
    for u in range(image_size):
        for v in range(image_size):
            x, y = u - center, v - center
            kernel[u,v] = np.sqrt(x**2 + y**2)
    return kernel

In [ ]:
def ideal_lpf(D0, image_size):
    ff = D(image_size) <= D0
    return ff


def ga_lpf(D0, image_size):
    ff = np.exp(-(D(image_size)**2) / (2*(D0**2)))
    return ff

# def ga_lpf(D0, image_size):
#    filter_function = np.exp(-(D(image_size=image_size)**2) / (2*D0**2))
#    return filter_function


def bu_lpf(D0, n, image_size):
    ff = 1/(1 + (D(image_size)/D0)**(2*n))
    return ff

In [ ]:
rgb_image = plt.imread('./images/lena.jpg')
gray_image = make_gray_image(rgb_image)

noisy_image = add_gaussian_noise(gray_image, 0, 25)




dft_noisy = np.fft.fft2(noisy_image)
dft_noisy = np.fft.fftshift(dft_noisy)

glpf = ga_lpf(40, noisy_image.shape[0])
ilpf = ideal_lpf(40, noisy_image.shape[0])
blpf = bu_lpf(40, 4, noisy_image.shape[0])


gfi = dft_noisy * glpf
ifi = dft_noisy * ilpf
bfi = dft_noisy * blpf


gfi1 = np.fft.ifftshift(gfi)
ifi1 = np.fft.ifftshift(ifi)
bfi1 = np.fft.ifftshift(bfi)

g = np.fft.ifft2(gfi1).real.astype(np.uint8)
i = np.fft.ifft2(ifi1).real.astype(np.uint8)
b = np.fft.ifft2(bfi1).real.astype(np.uint8)


plt.subplot(221)
plt.imshow(noisy_image, cmap='gray')
plt.subplot(222)
plt.imshow(g, cmap='gray')
plt.subplot(223)
plt.imshow(i, cmap='gray')
plt.subplot(224)
plt.imshow(b, cmap='gray')

plt.show()

In [ ]:
noisy_image